# Load models and get predictions

In [7]:
import joblib
import pandas as pd

In [8]:
# lgbmclassifier model 
loaded_model = joblib.load('models/lgbm_model.pkl')

In [1]:
sample_input = {
    'age': 55,
    'sex': 1,
    'chest_pain_type': 2,
    'resting_blood_pressure': 130,
    'cholesterol': 250,
    'fasting_blood_sugar': 0,
    'resting_electrocardiogram': 1,
    'max_heart_rate_achieved': 150,
    'exercise_induced_angina': 0,
    'st_depression': 1.0,
    'st_slope': 2,
    'num_major_vessels': 1,
    'thalassemia': 3
}

In [9]:
# Define the expected feature names (must match X_train from training)
expected_features = [
    'age', 'sex', 'chest_pain_type', 'resting_blood_pressure', 'cholesterol',
    'fasting_blood_sugar', 'resting_electrocardiogram', 'max_heart_rate_achieved',
    'exercise_induced_angina', 'st_depression', 'st_slope', 'num_major_vessels', 'thalassemia'
]

In [10]:
# Create a sample input for prediction
sample_input = {
    'age': 45,
    'sex': 1,
    'chest_pain_type': 1,
    'resting_blood_pressure': 130,
    'cholesterol': 150,
    'fasting_blood_sugar': 0,
    'resting_electrocardiogram': 1,
    'max_heart_rate_achieved': 50,
    'exercise_induced_angina': 0,
    'st_depression': 1.0,
    'st_slope': 2,
    'num_major_vessels': 1,
    'thalassemia': 3
}

In [11]:
# Convert sample input to DataFrame with correct feature names
sample_df = pd.DataFrame([sample_input], columns=expected_features)

In [12]:
prediction = loaded_model.predict(sample_df)

In [13]:
prediction_proba = loaded_model.predict_proba(sample_df)

In [14]:
print(f"Prediction: {'Disease' if prediction[0] == 1 else 'No Disease'}")
print(f"Prediction Probability: {prediction_proba[0]}")

Prediction: No Disease
Prediction Probability: [0.76924718 0.23075282]


Tuned cnn

In [2]:
import tensorflow as tf
import numpy as np
import os

In [5]:
# Define the path to the saved model
model_dir = 'models'  # Update if the model is saved elsewhere
model_path = os.path.join(model_dir, 'kaggle_best_cnn_tuned_model.keras')  # Use .keras, .h5, or savedmodel

In [6]:

# Load the model
try:
    model = tf.keras.models.load_model(model_path)
    print("Model loaded successfully.")
    model.summary()  # Print model architecture for verification
except Exception as e:
    raise RuntimeError(f"Failed to load model: {e}")

# Convert sample input to array in the correct order
feature_order = [
    'age', 'sex', 'chest_pain_type', 'resting_blood_pressure', 'cholesterol',
    'fasting_blood_sugar', 'resting_electrocardiogram', 'max_heart_rate_achieved',
    'exercise_induced_angina', 'st_depression', 'st_slope', 'num_major_vessels',
    'thalassemia'
]
input_array = np.array([sample_input[feature] for feature in feature_order], dtype=np.float32)

# Apply preprocessing (standardization)
# Placeholder: Typical mean and std for heart disease dataset (replace with your training data's values)
feature_means = np.array([54.0, 0.7, 2.3, 131.0, 246.0, 0.15, 0.5, 150.0, 0.33, 1.0, 1.6, 0.7, 2.3])
feature_stds = np.array([9.0, 0.5, 1.0, 17.5, 51.8, 0.36, 0.6, 23.0, 0.47, 1.2, 0.6, 0.9, 0.6])
input_array = (input_array - feature_means) / feature_stds

# Reshape for model input: (1, 13, 1)
input_array = input_array.reshape(1, 13, 1)

# Alternative: If you have a fitted scaler from training
"""
from joblib import load
scaler = load('path/to/scaler.joblib')  # Load your trained scaler
input_array = scaler.transform(input_array.reshape(1, -1)).reshape(1, 13, 1)
"""

# Make prediction
try:
    prediction = model.predict(input_array)
except Exception as e:
    raise RuntimeError(f"Prediction failed: {e}")

# Interpret the prediction (sigmoid output: probability of class 1)
probability = prediction[0][0]
class_label = 1 if probability >= 0.5 else 0
print(f"\nSample Input: {sample_input}")
print(f"Preprocessed Input: {input_array.flatten()}")
print(f"Predicted Probability: {probability:.4f}")
print(f"Predicted Class: {class_label} ({'Heart Disease' if class_label == 1 else 'No Heart Disease'})")

Model loaded successfully.


C:\Users\avish\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 10 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                      │ (None, 13, 48)              │             192 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d (MaxPooling1D)         │ (None, 6, 48)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_1 (Conv1D)                    │ (None, 6, 96)               │          13,920 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_1 (MaxPooling1D)       │ (None, 3, 96)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 3, 96)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 288)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │          36,992 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 102,468 (400.27 KB)

 Trainable params: 51,233 (200.13 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 51,235 (200.14 KB)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 586ms/step

Sample Input: {'age': 55, 'sex': 1, 'chest_pain_type': 2, 'resting_blood_pressure': 130, 'cholesterol': 250, 'fasting_blood_sugar': 0, 'resting_electrocardiogram': 1, 'max_heart_rate_achieved': 150, 'exercise_induced_angina': 0, 'st_depression': 1.0, 'st_slope': 2, 'num_major_vessels': 1, 'thalassemia': 3}
Preprocessed Input: [ 0.11111111  0.6        -0.3        -0.05714286  0.07722008 -0.41666667
  0.83333333  0.         -0.70212766  0.          0.66666667  0.33333333
  1.16666667]
Predicted Probability: 0.2259
Predicted Class: 0 (No Heart Disease)
